In [1]:
import re
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
date_list =[]
mae_list = []
smape_list = []
with open('E:\day-ahead price\Code\Time-Series-Library-main\Results\iso PatchTST/result_long_term_forecast.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()  # 读取所有行
    for i in range(len(lines)):
        line = lines[i].strip()  # 去除文本中的换行符
        index = line.find('2023-')  # 查找'2023-'在每一行中的位置
        if index == -1:
            index = line.find('2024-')
        index_dataset = line.find('iso')
        index_model = line.find('PatchTST')
        if index_dataset != -1 and index_model != -1:
            if index != -1:  # 如果找到了'2023-'
                result = line[index:index+10]  # 记录'2023-'及其后面的5个字符
                date_list.append(pd.to_datetime(result))
                if i+1 < len(lines):  # 如果存在下一行
                    next_line = lines[i+1].strip()  # 获取下一行内容
                    numbers = re.findall(r'\d+\.\d+|\d+', next_line)  # 使用正则表达式找出所有的数字，包括整数和小数
                    float_numbers = [float(num) for num in numbers]  # 将字符串转换为浮点数
                    mae_list.append(float_numbers[1])
                    smape_list.append(float_numbers[2])
df_result = pd.DataFrame({'date': date_list, 'mae': mae_list, 'smape': smape_list})
df_result

,date,mae,smape
0,2023-01-01,65.669243,0.851144
1,2023-01-02,31.621893,0.602266
2,2023-01-03,12.185302,0.324563
3,2023-01-04,14.767303,0.274758
4,2023-01-05,9.757164,0.247215
...,...,...,...
360,2023-12-27,2.624925,0.097926
361,2023-12-28,4.490261,0.164413
362,2023-12-29,4.377119,0.159459
363,2023-12-30,3.390560,0.116676


In [3]:
df_result['mae'].mean(), df_result['smape'].mean()

(9.751209779961469, 0.22473444028259956)

In [15]:
df_month = df_result[(pd.to_datetime('2023-01-01') <= df_result['date']) & (df_result['date'] <= pd.to_datetime('2023-04-15'))]
df_month['mae'].mean(), df_month['smape'].mean()

(15.860272895722161, 0.3218304595067388)

In [13]:
date_list =[]
mae_list = []
smape_list = []
with open('E:\day-ahead price\Code\Time-Series-Library-main\Time-Series-Library-main/result_long_term_forecast.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()  # 读取所有行
    for i in range(len(lines)):
        line = lines[i].strip()  # 去除文本中的换行符
        index = line.find('2023-')  # 查找'2023-'在每一行中的位置
        index_dataset = line.find('iso')
        index_model = line.find('VMD_LSTM')
        if index_dataset != -1 and index_model != -1:
            if index != -1:  # 如果找到了'2023-'
                result = line[index:index+10]  # 记录'2023-'及其后面的5个字符
                date_list.append(pd.to_datetime(result))
                if i+1 < len(lines):  # 如果存在下一行
                    next_line = lines[i+1].strip()  # 获取下一行内容
                    numbers = re.findall(r'\d+\.\d+|\d+', next_line)  # 使用正则表达式找出所有的数字，包括整数和小数
                    float_numbers = [float(num) for num in numbers]  # 将字符串转换为浮点数
                    mae_list.append(float_numbers[1])
                    smape_list.append(float_numbers[2])
df_result = pd.DataFrame({'date': date_list, 'mae': mae_list, 'smape': smape_list})
df_result

,date,mae,smape
0,2023-01-01,23.413263,0.468130
1,2023-01-02,30.004547,0.638614
2,2023-01-03,22.511314,0.452723
3,2023-01-04,17.195099,0.314778
4,2023-01-05,24.415579,0.463387
...,...,...,...
541,2023-09-26,23.025229,0.672551
542,2023-09-27,21.164961,0.586781
543,2023-09-28,23.708975,0.685329
544,2023-09-29,20.813932,0.583288


In [14]:
df_result = df_result[-273:]

In [15]:
df_result['mae'].mean(), df_result['smape'].mean()

(28.547044727828478, 0.6228666047910194)

In [21]:
df_month = df_result[(pd.to_datetime('2023-01-01') <= df_result['date']) & (df_result['date'] <= pd.to_datetime('2023-04-15'))]
df_month['mae'].mean(), df_month['smape'].mean()

(40.16138064066569, 0.6929721640689033)

In [9]:
date_list =[]
mae_list = []
smape_list = []
with open('E:\day-ahead price\Code\Time-Series-Library-main\Results\iso VMD_LSTM2/result_long_term_forecast.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()  # 读取所有行
    for i in range(len(lines)):
        line = lines[i].strip()  # 去除文本中的换行符
        index = line.find('2023-')  # 查找'2023-'在每一行中的位置
        index_dataset = line.find('iso')
        index_model = line.find('VMD_LSTM')
        if index_dataset != -1 and index_model != -1:
            if index != -1:  # 如果找到了'2023-'
                result = line[index:index+10]  # 记录'2023-'及其后面的5个字符
                date_list.append(pd.to_datetime(result))
                if i+1 < len(lines):  # 如果存在下一行
                    next_line = lines[i+1].strip()  # 获取下一行内容
                    numbers = re.findall(r'\d+\.\d+|\d+', next_line)  # 使用正则表达式找出所有的数字，包括整数和小数
                    float_numbers = [float(num) for num in numbers]  # 将字符串转换为浮点数
                    mae_list.append(float_numbers[1])
                    smape_list.append(float_numbers[2])
with open('E:\day-ahead price\Code\Time-Series-Library-main\Results extend\iso VMD_LSTM/result_long_term_forecast.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()  # 读取所有行
    for i in range(len(lines)):
        line = lines[i].strip()  # 去除文本中的换行符
        index = line.find('2023-')  # 查找'2023-'在每一行中的位置
        if index == -1:
            index = line.find('2024-')
        index_dataset = line.find('iso')
        index_model = line.find('VMD_LSTM')
        if index_dataset != -1 and index_model != -1:
            if index != -1:  # 如果找到了'2023-'
                result = line[index:index+10]  # 记录'2023-'及其后面的5个字符
                date_list.append(pd.to_datetime(result))
                if i+1 < len(lines):  # 如果存在下一行
                    next_line = lines[i+1].strip()  # 获取下一行内容
                    numbers = re.findall(r'\d+\.\d+|\d+', next_line)  # 使用正则表达式找出所有的数字，包括整数和小数
                    float_numbers = [float(num) for num in numbers]  # 将字符串转换为浮点数
                    mae_list.append(float_numbers[1])
                    smape_list.append(float_numbers[2])
df_result = pd.DataFrame({'date': date_list, 'mae': mae_list, 'smape': smape_list})
df_result.sort_values(by='date', inplace=True)
df_result.drop_duplicates(subset='date', keep='first', inplace=True)
# df_result = df_result[df_result['date'] >= pd.to_datetime('2023-04-01')]
df_result

,date,mae,smape
0,2023-01-01,23.413263,0.468130
274,2023-01-02,24.834755,0.561290
2,2023-01-03,22.511314,0.452723
3,2023-01-04,17.195099,0.314778
277,2023-01-05,18.880312,0.379605
...,...,...,...
633,2023-12-27,15.028839,0.463312
634,2023-12-28,14.900696,0.460813
635,2023-12-29,15.312531,0.476390
636,2023-12-30,12.479947,0.377259


In [10]:
df_result['mae'].mean(), df_result['smape'].mean()

(27.82065191660842, 0.6066624770834022)

capacity prediction

In [5]:
path = r'E:\day-ahead price\Code\Time-Series-Library-main\Results\capacity timesnet\results'
os.chdir(path)
files = os.listdir(path)
df = pd.DataFrame()
df_d3 = pd.DataFrame()
df_d4 = pd.DataFrame()
df_d5 = pd.DataFrame()
for file in files:
    # print(file)
    # find date in each file name
    index = file.find('2023-')
    date = file[index:index+10]
    # change directory to each file
    os.chdir(path + '\\' + file)
    # read each npy file
    data_pred = np.load('pred.npy', allow_pickle=True)
    data_true = np.load('true.npy', allow_pickle=True)
    # data_metric = np.load('metrics.npy', allow_pickle=True)
    # print(data_pred, data_true)
    data_true_d2 = data_true.reshape(-1)[1]
    data_pred_d2 = data_pred.reshape(-1)[1]
    data_pred_d3 = data_pred.reshape(-1)[2]
    data_pred_d4 = data_pred.reshape(-1)[3]
    data_pred_d5 = data_pred.reshape(-1)[4]
    # print(data_true_d2, data_pred_d2)
    # calculate mae and smape
    mae = np.mean(np.abs(data_true_d2 - data_pred_d2))
    smape = np.mean(np.abs(data_true_d2 - data_pred_d2) / ((np.abs(data_true_d2) + np.abs(data_pred_d2)) / 2))
    # print(mae, smape)
    df = df.append({'date': pd.to_datetime(date) - datetime.timedelta(3), 'mae': mae, 'smape': smape, 'pred': data_pred_d2, 'true': data_true_d2}, ignore_index=True)
    df_d3 = df_d3.append({'date': pd.to_datetime(date) - datetime.timedelta(2), 'pred': data_pred_d3}, ignore_index=True)
    df_d4 = df_d4.append({'date': pd.to_datetime(date) - datetime.timedelta(1), 'pred': data_pred_d4}, ignore_index=True)
    df_d5 = df_d5.append({'date': pd.to_datetime(date), 'pred': data_pred_d5}, ignore_index=True)
df
    

C:\Users\PC\AppData\Local\Temp\ipykernel_19188\3933643260.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': pd.to_datetime(date) - datetime.timedelta(3), 'mae': mae, 'smape': smape, 'pred': data_pred_d2, 'true': data_true_d2}, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_19188\3933643260.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_d3 = df_d3.append({'date': pd.to_datetime(date) - datetime.timedelta(2), 'pred': data_pred_d3}, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_19188\3933643260.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_d4 = df_d4.append({'date': pd.to_datetime(date) - datetime.timedelta(1), 'pred': data_pred_d4}, ignore_index=True)
C:

,date,mae,smape,pred,true
0,2023-03-29,284.753906,0.006752,42315.753906,42031.0
1,2023-03-30,3192.089844,0.078147,42443.089844,39251.0
2,2023-03-31,4992.808594,0.124517,42593.808594,37601.0
3,2023-04-01,9342.160156,0.250951,41898.160156,32556.0
4,2023-04-02,9776.203125,0.265917,41652.203125,31876.0
...,...,...,...,...,...
270,2023-12-24,236.324219,0.004821,49142.324219,48906.0
271,2023-12-25,236.328125,0.004821,49142.328125,48906.0
272,2023-12-26,236.324219,0.004821,49142.324219,48906.0
273,2023-12-27,236.324219,0.004821,49142.324219,48906.0


In [30]:
df_month = df[(pd.to_datetime('2023-03-01') <= df['date']) & (df['date'] <= pd.to_datetime('2023-12-31'))]
df_month['mae'].mean(), df_month['smape'].mean()

(1863.7028, 0.04832246603378979)

In [31]:
df.to_csv('E:\day-ahead price\Code\Time-Series-Library-main\Results\capacity timesnet\capacity_timesnet.csv', index=False)
df_d3.to_csv('E:\day-ahead price\Code\Time-Series-Library-main\Results\capacity timesnet\capacity_timesnet_d3.csv', index=False)
df_d4.to_csv('E:\day-ahead price\Code\Time-Series-Library-main\Results\capacity timesnet\capacity_timesnet_d4.csv', index=False)
df_d5.to_csv('E:\day-ahead price\Code\Time-Series-Library-main\Results\capacity timesnet\capacity_timesnet_d5.csv', index=False)

In [7]:
path = r'E:\day-ahead price\Code\Time-Series-Library-main\Results\iso capacity\results'
os.chdir(path)
files = os.listdir(path)
df = pd.DataFrame()
df_d2 = pd.DataFrame()

for file in files:
    # print(file)
    # find date in each file name
    index = file.find('2023-')
    if index == -1:
        index = file.find('2022-')
    date = file[index:index+10]
    # change directory to each file
    os.chdir(path + '\\' + file)
    # read each npy file
    data_pred = np.load('pred.npy', allow_pickle=True)
    data_true = np.load('true.npy', allow_pickle=True)
    # data_metric = np.load('metrics.npy', allow_pickle=True)
    # print(data_pred, data_true)
    data_true_d1 = data_true.reshape(-1)[0]
    data_pred_d1 = data_pred.reshape(-1)[0]
    data_pred_d2 = data_pred.reshape(-1)[1]
#     data_pred_d4 = data_pred.reshape(-1)[3]
#     data_pred_d5 = data_pred.reshape(-1)[4]
#     # print(data_true_d2, data_pred_d2)
    # calculate mae and smape
    mae = np.mean(np.abs(data_true_d1 - data_pred_d1))
    smape = np.mean(np.abs(data_true_d1 - data_pred_d1) / ((np.abs(data_true_d1) + np.abs(data_pred_d1)) / 2))
#     # print(mae, smape)
    df = df.append({'date': pd.to_datetime(date) - datetime.timedelta(1), 'mae': mae, 'smape': smape, 'pred': data_pred_d1, 'true': data_true_d1}, ignore_index=True)
    df_d2 = df_d2.append({'date': pd.to_datetime(date), 'pred': data_pred_d2}, ignore_index=True)

df

C:\Users\PC\AppData\Local\Temp\ipykernel_19188\1923537778.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': pd.to_datetime(date) - datetime.timedelta(1), 'mae': mae, 'smape': smape, 'pred': data_pred_d1, 'true': data_true_d1}, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_19188\1923537778.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_d2 = df_d2.append({'date': pd.to_datetime(date), 'pred': data_pred_d2}, ignore_index=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_19188\1923537778.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'date': pd.to_datetime(date) - datetime.timedelta(1), 'mae': mae, 'smape': smape, 'pred': data_pred_d1, 'true': data_true_d1}, i

,date,mae,smape,pred,true
0,2022-09-30,1735.238281,0.094876,19157.238281,17422.0
1,2022-10-01,2439.072266,0.140350,18598.072266,16159.0
2,2022-10-02,1298.134766,0.072773,18487.134766,17189.0
3,2022-10-03,1655.826172,0.094852,18284.826172,16629.0
4,2022-10-04,115.197266,0.006368,18032.802734,18148.0
...,...,...,...,...,...
360,2023-09-25,71.996094,0.004068,17735.996094,17664.0
361,2023-09-26,335.255859,0.018841,17626.744141,17962.0
362,2023-09-27,991.244141,0.057378,17771.244141,16780.0
363,2023-09-28,381.248047,0.022153,17400.248047,17019.0


In [9]:
df['mae'].mean(), df['smape'].mean()

(1085.6149, 0.053755434286657834)

In [8]:
df.to_csv('E:\day-ahead price\Code\Time-Series-Library-main\Results\iso capacity\capacity_timesnet.csv', index=False)
df_d2.to_csv('E:\day-ahead price\Code\Time-Series-Library-main\Results\iso capacity\capacity_timesnet_d2.csv', index=False)
